# Pengantar Rekayasa Prompt
Rekayasa prompt adalah proses merancang dan mengoptimalkan prompt untuk tugas pemrosesan bahasa alami. Proses ini melibatkan pemilihan prompt yang tepat, penyetelan parameter, dan evaluasi kinerjanya. Rekayasa prompt sangat penting untuk mencapai akurasi dan efisiensi tinggi pada model NLP. Pada bagian ini, kita akan membahas dasar-dasar rekayasa prompt menggunakan model OpenAI untuk eksplorasi.


### Latihan 1: Tokenisasi
Jelajahi Tokenisasi menggunakan tiktoken, tokenizer cepat open-source dari OpenAI
Lihat [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) untuk contoh lainnya.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Latihan 2: Validasi Pengaturan Kunci Github Models

Jalankan kode di bawah ini untuk memastikan bahwa endpoint Github Models Anda sudah diatur dengan benar. Kode ini hanya mencoba prompt sederhana dan memvalidasi hasilnya. Input `oh say can you see` seharusnya dilengkapi dengan sesuatu seperti `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Latihan 3: Rekayasa
Jelajahi apa yang terjadi ketika Anda meminta LLM untuk memberikan jawaban atas sebuah prompt tentang topik yang mungkin tidak ada, atau tentang topik yang mungkin tidak diketahui karena berada di luar dataset pra-pelatihannya (lebih baru). Lihat bagaimana responsnya berubah jika Anda mencoba prompt yang berbeda, atau model yang berbeda.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Latihan 4: Berbasis Instruksi
Gunakan variabel "text" untuk mengatur konten utama 
dan variabel "prompt" untuk memberikan instruksi yang berkaitan dengan konten utama tersebut.

Di sini kita meminta model untuk merangkum teks tersebut agar mudah dipahami oleh siswa kelas dua SD


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Latihan 5: Prompt Kompleks
Cobalah permintaan yang memiliki pesan system, user, dan assistant
System mengatur konteks untuk assistant
Pesan User & Assistant memberikan konteks percakapan beberapa giliran

Perhatikan bagaimana kepribadian assistant diatur menjadi "sarkastik" dalam konteks system.
Cobalah menggunakan konteks kepribadian yang berbeda. Atau coba rangkaian pesan input/output yang berbeda


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Latihan: Jelajahi Intuisi Anda
Contoh-contoh di atas memberikan pola yang bisa Anda gunakan untuk membuat prompt baru (sederhana, kompleks, instruksi, dll.) - cobalah membuat latihan lain untuk mengeksplorasi beberapa ide lain yang sudah kita bahas seperti contoh, petunjuk, dan lainnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemah profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
